# DAIN

Based on the [original Colab file](https://github.com/baowenbo/DAIN/issues/44) by btahir. 
and [Alpha](https://github.com/AlphaGit)

In [ ]:
# Clone DAIN sources
!git clone -b colab-compatibility --depth 1 https://github.com/AlphaGit/DAIN

In [1]:
! rm -R DAIN/frames

In [2]:
! mkdir DAIN/frames
! mkdir DAIN/frames/input_frames && mkdir DAIN/frames/output_frames

In [12]:
################# Required Configurations ############################

INPUT_FILEPATH = "video/original.mp4" #param{type:"string"}

OUTPUT_FILEPATH = "video/output.mp4" #param{type:"string"}

################# Optional configurations ############################

TARGET_FPS = 80 #param{type:"number"}

INPUTFRAMES_DIRPATH = 'frames/input_frames' #@param{type:"string"}

OUTPUTFRAMES_DIRPATH = 'frames/output_frames' #@param{type:"string"}

#DAIN frames are a bit "shifted / smaller" compared to original input frames. This can partly be mitigated with resizing DAIN frames to the resolution +2px and cropping the result to the original resoultion with the starting point (1,1). Without this fix, DAIN tends to make "vibrating" output and it is pretty noticible with static elements like text.
RESIZE_HOTFIX = True #@param{type:"boolean"}

In [4]:
# Check your current GPU

# 16GB: Can handle 720p. 1080p will procude an out-of-memory error. 
# 8GB: Can handle 480p. 720p will produce an out-of-memory error.

!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Quadro P5000, 450.36.06, 16278 MiB


# Install dependencies.

This next step may take somewhere between 15-20 minutes.

In [ ]:
!apt-get update

In [ ]:
!apt-get install imagemagick imagemagick-doc libgl1-mesa-glx ffmpeg

In [ ]:
!pip install --upgrade pip

In [ ]:
# Install known used versions of PyTorch and SciPy
!pip install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip install scipy==1.1.0

In [ ]:
!pip install opencv-python
!pip install imageio

In [ ]:
# This takes a while. Just wait. ~15 minutes.
# Building DAIN.
%cd DAIN/my_package/
!./build.sh
print("Building #1 done.")
%cd ../../

In [ ]:
# Wait again. ~5 minutes.
# Building DAIN PyTorch correlation package.
%cd DAIN/PWCNet/correlation_package_pytorch1_0
!./build.sh
print("Building #2 done.")
%cd ../../..

In [ ]:
# Downloading pre-trained model
!mkdir DAIN/model_weights
!wget -O DAIN/model_weights/best.pth http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

In [5]:
!CUDA_VISIBLE_DEVICES=0
print("Finished installing dependencies.")

Finished installing dependencies.


# Start

In [5]:
import os
import cv2
import subprocess as sp
from tqdm import tqdm

In [6]:
# Detecting FPS of input file.

filename = os.path.basename(INPUT_FILEPATH)

cap = cv2.VideoCapture(f'video/{filename}')

fps = cap.get(cv2.CAP_PROP_FPS)

if(fps/TARGET_FPS>0.5):
  print("Define a higher fps, because there is not enough time for new frames. (Old FPS)/(New FPS) should be lower than 0.5. Interpolation will fail if you try.")

In [7]:
# ffmpeg extract - Generating individual frame PNGs from the source file.
! ffmpeg -i 'video/{filename}' 'DAIN/{INPUTFRAMES_DIRPATH}/%05d.png'

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [8]:
pngs_generated_count = int(sp.getoutput('ls '+'DAIN/'+INPUTFRAMES_DIRPATH+' | wc -l'))

In [9]:
pngs_generated_count

1625

In [10]:
%cd DAIN/

import shutil

print(f"Input FPS: {fps}")
print(f"{pngs_generated_count} frame PNGs generated.")

# Checking if PNG do have alpha
%cd {INPUTFRAMES_DIRPATH}
channels = sp.getoutput('identify -format %[channels] 00001.png')
print (f"{channels} detected")

# Removing alpha if detected
if "a" in channels:
  print("Alpha detected and will be removed.")
  print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))

%cd ../../

/notebooks/DAIN
Input FPS: 23.98
1625 frame PNGs generated.
/notebooks/DAIN/frames/input_frames
srgb detected
/notebooks/DAIN


In [11]:
# Interpolation

In [13]:
!python -W ignore colab_interpolate.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --start_frame 1 --end_frame {pngs_generated_count} --frame_input_dir '{INPUTFRAMES_DIRPATH}' --frame_output_dir '{OUTPUTFRAMES_DIRPATH}'

revise the unique id to a random numer 58035
Namespace(SAVED_MODEL=None, alpha=[0.0, 1.0], arg='./model_weights/58035-Sun-Sep-20-23-38/args.txt', batch_size=1, channels=3, ctx_lr_coe=1.0, datasetName='Vimeo_90K_interp', datasetPath='', dataset_split=97, debug=False, depth_lr_coe=0.001, dtype=<class 'torch.cuda.FloatTensor'>, end_frame=1625, epsilon=1e-06, factor=0.2, filter_lr_coe=1.0, filter_size=4, flow_lr_coe=0.01, force=False, frame_input_dir='frames/input_frames', frame_output_dir='frames/output_frames', log='./model_weights/58035-Sun-Sep-20-23-38/log.txt', lr=0.002, netName='DAIN_slowmotion', no_date=False, numEpoch=100, occ_lr_coe=1.0, patience=5, rectify_lr=0.001, save_path='./model_weights/58035-Sun-Sep-20-23-38', save_which=1, seed=1, start_frame=1, time_step=0.29975, uid=None, use_cuda=True, use_cudnn=1, weight_decay=0, workers=8)
cudnn is used
Interpolate 2 frames
100%|█████████████████████████████████████| 1624/1624 [1:07:03<00:00,  2.48s/it]
Finished processing images.


In [14]:
# Finding DAIN Frames, upscaling and cropping to match original
import numpy as np
if(RESIZE_HOTFIX==True):
  images = []
  for filename in tqdm(os.listdir(f'{OUTPUTFRAMES_DIRPATH}')):
    img = cv2.imread(os.path.join(f'{OUTPUTFRAMES_DIRPATH}',filename))
    part_filename = os.path.splitext(filename)
    if(part_filename[0].endswith('0')==False):
      dimension = (img.shape[1]+2, img.shape[0]+2)
      resized = cv2.resize(img, dimension, interpolation=cv2.INTER_LANCZOS4)
      crop = resized[1:(dimension[1]-1), 1:(dimension[0]-1)]
      ns = str(f'{OUTPUTFRAMES_DIRPATH}/')+part_filename[0]+".png"
      cv2.imwrite(ns, crop)

100%|██████████| 4873/4873 [01:38<00:00, 49.54it/s] 


In [18]:
%cd {OUTPUTFRAMES_DIRPATH}
! ffmpeg -i '../../../{INPUT_FILEPATH}' -acodec copy '../../../video/temp_outputaudio.aac'
#print("\n")
#! ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' -i '../../../video/temp_outputaudio.aac' -shortest -c:v copy -c:a aac -b:a 256k  '../../../{OUTPUT_FILEPATH}'
#! rm temp_outputaudio.aac
%cd ../..

/notebooks/DAIN/frames/output_frames
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-

In [25]:
def img2video(read_path,save_path):
    #print(read_path)
    #print(save_path)
    files = []
    for r, _, f in os.walk(read_path):
        for file in f:
            if '.png' in file:
                files.append([os.path.join(r, file),file.replace(".png","").replace("-checkpoint","")])
    #print("")
    files.sort(key=lambda x: int(x[1]))
    img = cv2.imread(files[0][0])
    height, width, _ = img.shape
    size = (width,height)
    #print("VidSize: ",str(size))
    #print("")
    #-----------------------------------------------
    out = cv2.VideoWriter(save_path,cv2.VideoWriter_fourcc(*'mp4v'), 60, size)

    for file in tqdm(range(len(files))):
        path = files[file][0]
        img = cv2.imread(path)
        out.write(img)
    out.release()

In [27]:
img2video(OUTPUTFRAMES_DIRPATH,'../'+OUTPUT_FILEPATH)

100%|██████████| 4873/4873 [00:47<00:00, 102.85it/s]


In [28]:
from IPython.display import Video as V1
print(INPUT_FILEPATH)
V1('../'+str(INPUT_FILEPATH))

video/original.mp4


In [29]:
from IPython.display import Video as V2
print(OUTPUT_FILEPATH)
V2('../'+OUTPUT_FILEPATH)

video/output.mp4
